## 1. Parte 2: Datos cuantitativos de compañía de envíos

<p>Continuando con el ejemplo aplicado de la compañía de envíos. En este documento vamos a explorar los métodos para el resumen de datos cuantitativos.</p>

<p>Tópicos a abordar en este tutorial:</p>

<ul>
    <li>Atributos y variables.</li>
    <li>Tipos de variables.</li>
    <li>Distribuciones de frecuencia.</li>
</ul>

<p>Inicialmente deberemos importar las librerías necesarias para realizar nuestro análisis:</p>

In [1]:
import numpy as np
import pandas as pd
import math

<p>Luego de importar las librerías procederemos a hacer la lectura de un dataset, el archivo a cargar se denomina <code>UPDI_CLEAN.csv</code>, el cual es un archivo de tipo <code>.csv</code> (Comma Separated Values, o Valores Separados por Comas), resultado de la limpieza y manipulación de datos que se hizo en el ejercicio anterior. Por tanto, volveremos a utilizar pandas como <code>pd</code> y el comando <code>.read_csv()</code> para leer el contenido del dataset:</p>

In [2]:
df0 = pd.read_csv('DATASETS/UPDI_CLEAN.csv', index_col='FID') # index_col se utiliza cuando se requiere utilizar 
                                                    # una columna como índice, en este caso 'FID'
df0.head()

,TYPE,CITY,STATE,ZIP,LATITUDE,LONGITUDE
FID,,,,,,
1,Self-Service,Tampa,FL,33605,27.956644,-82.439072
2,Self-Service,Tampa,FL,33606,27.941708,-82.465888
3,Self-Service,Tampa,FL,33603,27.994389,-82.459439
4,Authorized Ship Center,Tampa,FL,33604,28.024894,-82.484288
5,Self-Service,Tampa,FL,33607,27.976984,-82.484789


## 2. Variables cuantitativas y tablas de frecuencias:

### 2.1. Variable cuantitativa - Latitud:

<p>Ahora procederemos a tabular las variables cuantitativas del set de datos. La primera es la latitud (como habíamos visto en el ejercicio anterior), por tal motivo lo primero que deberemos realizar es calcular el rango de la variable para conocer qué tan dispersos se encuentran los datos y poder empezar a determinar los intervalos de la tabla de frecuencias agrupadas. Lo anterior quiere decir que ya no se usarán etiquetas o categorías como <code>Self-service</code> o <code>NY</code> sino que procederemos a utilizar intervalos numéricos. A continuación, se determinan los valores mínimo, máximo, y se calcula el rango:</p>

In [3]:
la_min = round(df0['LATITUDE'].min(),0)
print("El valor mínimo de latitud es: " + str(la_min))
la_max = round(df0['LATITUDE'].max(),0)
print("El valor máximo de latitud es: " + str(la_max))
la_range = round(la_max - la_min,2)
print("El rango calculado para latitud es: " + str(la_range))

El valor mínimo de latitud es: 19.0
El valor máximo de latitud es: 65.0
El rango calculado para latitud es: 46.0


<p>Posteriormente se calcula el número de intervalos (o clases) que tendrá la tabla de frecuencias teniendo en cuenta la Regla de Sturges:</p>

In [4]:
n = df0['LATITUDE'].count() # Tamaño de la muestra

factor = 3.322 # factor que es igual (=) a 1/Log(2)

la_k = 1 + (factor*np.log10(n)) # Número de intervalos o clases
print("El número de intervalos calculado es: " + str(la_k))

la_k_rounded = int(math.ceil(la_k)) # Número de intervalos redondeados al entero superior
print("El número de intervalos redondeado al entero superior es: " + str(la_k_rounded))

El número de intervalos calculado es: 16.57555136046842
El número de intervalos redondeado al entero superior es: 17


<p>Luego de calcular el número de intervalos sugeridos para la tabla de frecuencias, entonces se procede a determinar la amplitud de los intervalos, dada por el cociente entre el rango y el número de intervalos:</p>

In [5]:
la_c = la_range/la_k_rounded # Amplitud del intervalo
print("La amplitud del intervalo es: " + str(la_c))

la_c_rounded = int(math.ceil(la_c)) # Amplitud del intervalo redondeado al enterio superior
print("La amplitud del intervalo redondeada al entero superior es: " + str(la_c_rounded))

La amplitud del intervalo es: 2.7058823529411766
La amplitud del intervalo redondeada al entero superior es: 3


<p>Ya teniendo la amplitud del intervalo, se actualiza el valor del rango y el valor mínimo de la siguiente forma:</p>

<ul>
    <li>El <b>nuevo rango</b> (<code>la_n_range</code>) será el producto de la amplitud y el número de intervalos redondeados al entero superior.</li>
    <li>La <b>diferencia de rangos</b> (<code>la_diff_range</code>) será la resta entre el nuevo rango y el rango calculado inicialmente.</li>
    <li>El <b>nuevo valor mínimo</b> (<code>la_n_min</code>) será la diferencia del valor mínimo y la diferencia de rangos dividida entre dos (2) y redondeada al entero inferior.</li>
</ul>

<p>Lo anterior se hace con el fin de ampliar un poco la cobertura de los intervalos y asegurar que ningún valor quede por fuera de los mismos.</p>

In [6]:
la_n_range = la_c_rounded*la_k_rounded # Nuevo rango
print("El nuevo rango calculado es: " + str(la_n_range))

la_diff_range = round(la_n_range - la_range,2) # Diferencia entre rangos
print("La diferencia entre rangos es: " + str(la_diff_range))

la_n_min = la_min - math.floor(la_diff_range/2) # Nuevo valor mínimo
print("El nuevo valor mínimo es: " + str(la_n_min))

El nuevo rango calculado es: 51
La diferencia entre rangos es: 5.0
El nuevo valor mínimo es: 17.0


<p>Ya tenemos el nuevo valor mínimo, el número de intervalos y la amplitud del intervalo. Ahora podemos determinar cada uno de los intervalos de la tabla de frecuencia acumulada de la siguiente forma:</p>

<ol>
1. Se crea una lista y un diccionario, ambos vacíos, para almacenar los intervalos:
</ol>

In [7]:
la_intervals = [] # La lista la utilizaremos para crear una nueva columna con los intervalos según el valor de la columna 'LATITUDE'
la_dict_intervals = {} # El diccionario lo utilizaremos para crear un histograma de frecuencias y visualizar la distribución

<ol>
2. a) Comenzaremos creando una función que incluye un ciclo 'for' desde i = 0 hasta el número de intervalos calculados ('range(la_k_rounded)'). No importa que 'range()' no incluya el último valor ya que nos interesa que el ciclo se repita el número de veces equivalente al número de intervalos, y esto se logra porque la función inicia desde cero (0), como en el siguiente ejemplo:
</ol>

In [8]:
c_prueba = [i for i in range(14)] # cada uno de los valores i en el rango(14)
print(c_prueba) # Se imprimen los valores
print(len(c_prueba)) # Se imprime la cantidad de valores almacenados

# Nota: A pesar que la función 'range(14)' no incluye el valor '14' como tal en el listado, de todas formas se generan 14 valores.

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
14


<ol>
2. b) Por tanto, procedemos a crear la función que llamaremos 'clases':
</ol>

In [9]:
def clases(n_min, k_rounded, c_rounded, intervals, dict_intervals):
    
    d = 0
    
    for i in range(k_rounded):

        interval_j = [] # Se crea una lista interna que almacenará un (1) intervalo cada vez que el ciclo se repita

        if d == 0: # Cuando 'd' es igual a cero (0) se debe hacer lo siguiente:

            low_lim = n_min # Limite inferior del intervalo será igual al 'nuevo valor mínimo'
            up_lim = low_lim + c_rounded # Límite superior del intervalo será igual al límite inferior más (+) la amplitud del intervalo

            interval_j.append(round(low_lim,3)) # Se añade el límite inferior (redondeado a 3 cifras) al listado interno
            interval_j.append(round(up_lim,3)) # Se añade el límite inferiro (redondeado a 3 cifras) al listado interno

            d = d + 1 # Se actualiza el valor del contador sumando (+ 1)

        elif d <= (k_rounded - 1): # Cuando 'd' es mayor a 0 y menor o igual al número de inervalos menos (-) uno (1),
            # es decir 16, entonces se debe hacer lo siguiente:

            low_lim = n_min + d*c_rounded # Límite inferior será igual al 'nuevo valor mínimo' más (+) 'd' veces la amplitud del intervalo
            up_lim = low_lim + c_rounded # Límite superior será igual a límite inferior actual más (+) la amplitud del intervalo

            interval_j.append(round(low_lim,3)) # Se añade el límite inferior (redondeado a 3 cifras) al listado interno
            interval_j.append(round(up_lim,3)) # Se añade el límite superior (redondeado a 3 cifras) al listado interno

            d = d + 1 # También se actualiza el valor del contador sumando (+ 1)

        else: # Cuando 'd' es mayor a 16 se rompe el ciclo
            break
            
        intervals.append(interval_j)# Se añade el listado interno (intervalo) al listado externo    
        dict_intervals[i+1] = interval_j # Se añade el listado interno (intervalo) al diccionario externo, inicia en i + 1 para
                                         # almacenar las llaves con valores de 1 a 17 en el diccionario
        
        # Se repite el ciclo hasta 'range(la_k_rounded)''
        
    print("El listado de intervalos determinado es: " + str(intervals)) # Se imprime el listado de intervalos
    print("El diccionario de intervalos creado es: " + str(dict_intervals)) # Se imprime el diccionario de intervalos
    
clases(la_n_min, la_k_rounded, la_c_rounded, la_intervals, la_dict_intervals)

El listado de intervalos determinado es: [[17.0, 20.0], [20.0, 23.0], [23.0, 26.0], [26.0, 29.0], [29.0, 32.0], [32.0, 35.0], [35.0, 38.0], [38.0, 41.0], [41.0, 44.0], [44.0, 47.0], [47.0, 50.0], [50.0, 53.0], [53.0, 56.0], [56.0, 59.0], [59.0, 62.0], [62.0, 65.0], [65.0, 68.0]]
El diccionario de intervalos creado es: {1: [17.0, 20.0], 2: [20.0, 23.0], 3: [23.0, 26.0], 4: [26.0, 29.0], 5: [29.0, 32.0], 6: [32.0, 35.0], 7: [35.0, 38.0], 8: [38.0, 41.0], 9: [41.0, 44.0], 10: [44.0, 47.0], 11: [47.0, 50.0], 12: [50.0, 53.0], 13: [53.0, 56.0], 14: [56.0, 59.0], 15: [59.0, 62.0], 16: [62.0, 65.0], 17: [65.0, 68.0]}


In [10]:
la_intervals[0:17]

[[17.0, 20.0],
 [20.0, 23.0],
 [23.0, 26.0],
 [26.0, 29.0],
 [29.0, 32.0],
 [32.0, 35.0],
 [35.0, 38.0],
 [38.0, 41.0],
 [41.0, 44.0],
 [44.0, 47.0],
 [47.0, 50.0],
 [50.0, 53.0],
 [53.0, 56.0],
 [56.0, 59.0],
 [59.0, 62.0],
 [62.0, 65.0],
 [65.0, 68.0]]

<p>Ya habiendo determinado los intervalos de la tabla de frecuencia, es momento de empezar a darle forma a dicha tabla. Lo primero que debemos hacer es crear una nueva columna que permita clasificar cada uno de los valores de la columna <code>LATITUDE</code> con el fin de determinar a qué intervalo pertenece cada valor:</p>

In [11]:
# Se crea una función 'la_cat' que recibirá un valor 'x' como argumento
def la_cat(x):
    for i in range(la_k_rounded): # Se inicia el ciclo for desde i = 0 hasta el número de intervalos ('la_k_rounded')
        for j in range(1):
            if i <= (la_k_rounded - 1): # Cuando i es menor o igual al número de intervalos menos (-) uno (1), es decir 16.
                
                if la_intervals[i][j] <= x < la_intervals[i][j+1]: # Si el valor 'x' se encuentra dentro del intervalo, entonces:
                    return (la_intervals[i][j], la_intervals[i][j+1]) # Se retorna el intervalo como una tupla '()'
            
            else: # Cuando i es igual a 'la_k_rounded', es decir estamos en el último intervalo
                
                if la_intervals[i][j] <= x <= la_intervals[i][j+1]: # Si el valor 'x' se encuentra dentro de dicho intervalo, entonces:
                    return (la_intervals[i][j], la_intervals[i][j+1]) # Se retorna el intervalo como una tupla '()'

df0["la_group"] = df0["LATITUDE"].apply(la_cat) # Se aplica la función 'la_cat' a cada valor de la columna 'LATITUDE' (x) y 
                                                # Se asigna cada intervalo resultante a la nueva columna 'la_group'

print(df0.tail(10)) # Se imprime la cola del dataframe

                         TYPE          CITY STATE   ZIP   LATITUDE  LONGITUDE  \
FID                                                                             
48812            Self-Service       Houlton    ME  4730  46.125586 -67.842154   
48813  Authorized Ship Center       Houlton    ME  4730  46.150701 -67.840403   
48814            Self-Service       Houlton    ME  4730  46.125755 -67.836195   
48815            Self-Service       Machias    ME  4654  44.715236 -67.457781   
48816            Self-Service     Mars Hill    ME  4758  46.514267 -67.866919   
48817                 Staffed  Presque Isle    ME  4769  46.699644 -68.026354   
48818            Self-Service  Presque Isle    ME  4769  46.681196 -68.015464   
48819  Authorized Ship Center  Presque Isle    ME  4769  46.688198 -68.010493   
48820            Self-Service       Caribou    ME  4736  46.860595 -68.008336   
48821  Authorized Ship Center     Fort Kent    ME  4743  47.250349 -68.596782   

           la_group  
FID  

In [12]:
count_null_group = df0["la_group"].isnull().sum()
count_null_group # Se verifica que no existan valores nulos dentro de la nueva columna creada

0

<p>Luego de crear la nueva columna <code>la_group</code> entonces manipulamos el dataframe y lo agrupamos con el método <code>.groupby()</code>, introduciendo como argumento el nombre de la columna y como método agregado <code>.count()</code> para que contar los valores dentro de cada intervalo:</p>

In [13]:
df0_gbla = df0[["LATITUDE", "la_group"]].groupby("la_group").count()
df0_gbla.columns = ["la_fi"]
df0_gbla

,la_fi
la_group,
"(17.0, 20.0)",15
"(20.0, 23.0)",102
"(23.0, 26.0)",511
"(26.0, 29.0)",2947
"(29.0, 32.0)",3770
"(32.0, 35.0)",9729
"(35.0, 38.0)",6636
"(38.0, 41.0)",13251
"(41.0, 44.0)",9132


<p>Luego de tener la frecuencia absoluta de cada intervalo, se procede a crear un nuevo atributo o columna llamado <code>la_mean</code> donde se almacenará el promedio de valores de cada intervalo, esto hará las veces de la marca de clase:</p>

In [14]:
df0_gbla["la_class"] = round(df0[["LATITUDE", "la_group"]].groupby("la_group").mean(),2)
df0_gbla

,la_fi,la_class
la_group,,
"(17.0, 20.0)",15,19.68
"(20.0, 23.0)",102,21.24
"(23.0, 26.0)",511,25.72
"(26.0, 29.0)",2947,27.39
"(29.0, 32.0)",3770,30.20
"(32.0, 35.0)",9729,33.58
"(35.0, 38.0)",6636,36.45
"(38.0, 41.0)",13251,39.74
"(41.0, 44.0)",9132,42.16


<p>Después de crear la marca de clase, entonces podemos crear la columna <code>la_hi</code> que almacenará la frecuencia relativa de los valores de la tabla:</p>

In [15]:
la_total_count = df0_gbla["la_fi"].sum()
df0_gbla["la_hi"] = round(df0_gbla["la_fi"]/la_total_count,4)
df0_gbla

,la_fi,la_class,la_hi
la_group,,,
"(17.0, 20.0)",15,19.68,0.0003
"(20.0, 23.0)",102,21.24,0.0021
"(23.0, 26.0)",511,25.72,0.0105
"(26.0, 29.0)",2947,27.39,0.0604
"(29.0, 32.0)",3770,30.20,0.0772
"(32.0, 35.0)",9729,33.58,0.1993
"(35.0, 38.0)",6636,36.45,0.1359
"(38.0, 41.0)",13251,39.74,0.2714
"(41.0, 44.0)",9132,42.16,0.1871


<p>Seguidamente se procede a calcular la frecuencia absoluta acumulada (<code>la_Fi</code>) que se refiere a la suma de los valores de frecuencia absoluta a medida que se desciende en la tabla:</p>

In [16]:
df0_gbla["la_Fi"] = df0_gbla["la_fi"].cumsum()
df0_gbla

,la_fi,la_class,la_hi,la_Fi
la_group,,,,
"(17.0, 20.0)",15,19.68,0.0003,15
"(20.0, 23.0)",102,21.24,0.0021,117
"(23.0, 26.0)",511,25.72,0.0105,628
"(26.0, 29.0)",2947,27.39,0.0604,3575
"(29.0, 32.0)",3770,30.20,0.0772,7345
"(32.0, 35.0)",9729,33.58,0.1993,17074
"(35.0, 38.0)",6636,36.45,0.1359,23710
"(38.0, 41.0)",13251,39.74,0.2714,36961
"(41.0, 44.0)",9132,42.16,0.1871,46093


<p>Lo mismo haremos con la frecuencia relativa, procederemos a calcular la columna acumulada (<code>la_Hi</code>):</p>

In [17]:
df0_gbla["la_Hi"] = df0_gbla["la_hi"].cumsum()
df0_gbla

,la_fi,la_class,la_hi,la_Fi,la_Hi
la_group,,,,,
"(17.0, 20.0)",15,19.68,0.0003,15,0.0003
"(20.0, 23.0)",102,21.24,0.0021,117,0.0024
"(23.0, 26.0)",511,25.72,0.0105,628,0.0129
"(26.0, 29.0)",2947,27.39,0.0604,3575,0.0733
"(29.0, 32.0)",3770,30.20,0.0772,7345,0.1505
"(32.0, 35.0)",9729,33.58,0.1993,17074,0.3498
"(35.0, 38.0)",6636,36.45,0.1359,23710,0.4857
"(38.0, 41.0)",13251,39.74,0.2714,36961,0.7571
"(41.0, 44.0)",9132,42.16,0.1871,46093,0.9442


<p>Con esta tabla es posible determinar, por ejemplo, que una gran parte de las instalaciones se encuentran ubicadas dentro del rango de latitud de <code>(38.0, 41.0)</code>, es decir, 13.251 instalaciones están ubicadas en una latitud mayor o igual a 38.0 y menor que 41.0 según la columna <code>la_fi</code>, y eso es equivalente a aproximadamente 27.14% del total de instalaciones de la compañía (columna <code>la_hi</code>). También es posible concluir que, dada la columna <code>la_Fi</code>, 36.961 instalaciones se encuentran localizadas en una latitud mayor o igual a 23.0 y menor a 41.0, eso es equivalente al 75.71% de las instalaciones (columna <code>la_Hi</code>).</p>

<p>Por lo tanto, la tabla de frecuencias acumulada resulta útil para realizar diferentes observaciones con respecto a la distribución de los datos que se encuentran en la columna <code>LATITUDE</code> y podemos ver el comportamiento de dicha variable y donde se centran dichos datos. Por ejemplo, si quisiéramos conocer qué porcentaje de instalaciones se encuentran en una latitud mayor o igual a 26.0 y menor que 50.0, entonces podríamos hacer lo siguiente:</p>

In [18]:
la_26_50 = (df0_gbla.loc[(26.0, 29.0):(47.0, 50.0),['la_hi']].sum())*100 # Se secciona el índice con valores desde 26 hasta 50
                                                                         # Y se cruzan con la columna 'la_hi'
print("El porcentaje (%) de instalaciones cuya latitud está entre 26.0 y 50.0 es igual a: " + str(la_26_50))

El porcentaje (%) de instalaciones cuya latitud está entre 26.0 y 50.0 es igual a: la_hi    98.61
dtype: float64


<p>Adicionalmente, si quisiéramos calcular el promedio de la variable <code>LONGITUDE</code> solamente es necesario multiplicar cada valor de la marca de clase (<code>la_class</code>) con la frecuencia relativa (<code>la_hi</code>) y sumar los productos:</p>

In [19]:
la_mean = (df0_gbla["la_class"]*df0_gbla["la_hi"]).sum()
la_mean

37.205324000000005

<p>Lo que quiere decir que las instalaciones tienden a estar localizadas en una latitud promedio aproximada de 37,21.</p>

### 2.2. Variable cuantitativa - Longitud:

<p>Seguiremos el mismo procedimiento para crear la tabla de frecuencias agrupadas con la variable <code>LONGITUDE</code> que almacena la coordenada longitud de las instalaciones de la compañía:</p>

<ol>
1. Se calculan los parámetros necesarios para determinar las clases o intervalos:
</ol>

In [20]:
log_min = round(df0["LONGITUDE"].min(),0)
print("El valor mínimo de longitud es: " + str(log_min))
log_max = round(df0["LONGITUDE"].max(),0)
print("El valor máximo de longitud es: " + str(log_max))
log_range = round(log_max - log_min,0)
print("El rango de valores para la longitud es: " + str(log_range))

n = df0["LONGITUDE"].count()
factor = 3.322
log_k = 1 + (factor*np.log10(n))
print("El número de intervalos determinado es: " + str(log_k))

log_k_rounded = int(math.ceil(log_k))
print("El número de intervalos redondeado al entero superior es: " + str(log_k_rounded))

log_c = log_range/log_k_rounded
print("La amplitud del intervalo es: " +str(log_c))

log_c_rounded = int(math.ceil(log_c))
print("La amplitud del intervalo redondeada al entero superior es: " + str(log_c_rounded))

log_n_range = log_c_rounded*log_k_rounded
print("El nuevo rango de valores es: " + str(log_n_range))

log_diff_range = round(log_n_range - log_range,2)
print("La diferencia entre rangos es: " + str(log_diff_range))

log_n_min = log_min - math.floor(log_diff_range/2)
print("El nuevo valor mínimo es: " + str(log_n_min))

El valor mínimo de longitud es: -160.0
El valor máximo de longitud es: -67.0
El rango de valores para la longitud es: 93.0
El número de intervalos determinado es: 16.57555136046842
El número de intervalos redondeado al entero superior es: 17
La amplitud del intervalo es: 5.470588235294118
La amplitud del intervalo redondeada al entero superior es: 6
El nuevo rango de valores es: 102
La diferencia entre rangos es: 9.0
El nuevo valor mínimo es: -164.0


<ol>
    2. Se aplica la función definida anteriormente (<code>clases</code>) para determinar los intervalos de la variable:
</ol>

In [21]:
log_intervals = []
log_dict_intervals = {}

clases(log_n_min, log_k_rounded, log_c_rounded, log_intervals, log_dict_intervals)

El listado de intervalos determinado es: [[-164.0, -158.0], [-158.0, -152.0], [-152.0, -146.0], [-146.0, -140.0], [-140.0, -134.0], [-134.0, -128.0], [-128.0, -122.0], [-122.0, -116.0], [-116.0, -110.0], [-110.0, -104.0], [-104.0, -98.0], [-98.0, -92.0], [-92.0, -86.0], [-86.0, -80.0], [-80.0, -74.0], [-74.0, -68.0], [-68.0, -62.0]]
El diccionario de intervalos creado es: {1: [-164.0, -158.0], 2: [-158.0, -152.0], 3: [-152.0, -146.0], 4: [-146.0, -140.0], 5: [-140.0, -134.0], 6: [-134.0, -128.0], 7: [-128.0, -122.0], 8: [-122.0, -116.0], 9: [-116.0, -110.0], 10: [-110.0, -104.0], 11: [-104.0, -98.0], 12: [-98.0, -92.0], 13: [-92.0, -86.0], 14: [-86.0, -80.0], 15: [-80.0, -74.0], 16: [-74.0, -68.0], 17: [-68.0, -62.0]}


<ol>
    3. Se crea una nueva columna que almacene los intervalos o clases dependiendo del valor de la variable evaluada (<code>LONGITUDE</code>):
</ol>

In [22]:
def log_cat(x):
    for i in range(log_k_rounded):
        for j in range(1):
            if i <= (log_k_rounded - 1):
                if log_intervals[i][j] <= x < log_intervals[i][j+1]:
                    return (log_intervals[i][j], log_intervals[i][j+1])
                else:
                    if log_intervals[i][j] <= x <= log_intervals[i][j+1]:
                        return (log_intervals[i][j], log_intervals[i][j+1])

df0["log_group"] = df0["LONGITUDE"].apply(log_cat)
print(df0.tail(10))

                         TYPE          CITY STATE   ZIP   LATITUDE  LONGITUDE  \
FID                                                                             
48812            Self-Service       Houlton    ME  4730  46.125586 -67.842154   
48813  Authorized Ship Center       Houlton    ME  4730  46.150701 -67.840403   
48814            Self-Service       Houlton    ME  4730  46.125755 -67.836195   
48815            Self-Service       Machias    ME  4654  44.715236 -67.457781   
48816            Self-Service     Mars Hill    ME  4758  46.514267 -67.866919   
48817                 Staffed  Presque Isle    ME  4769  46.699644 -68.026354   
48818            Self-Service  Presque Isle    ME  4769  46.681196 -68.015464   
48819  Authorized Ship Center  Presque Isle    ME  4769  46.688198 -68.010493   
48820            Self-Service       Caribou    ME  4736  46.860595 -68.008336   
48821  Authorized Ship Center     Fort Kent    ME  4743  47.250349 -68.596782   

           la_group       l

In [23]:
count_null_group = df0["log_group"].isnull().sum()
count_null_group # Se verifica que no existan valores nulos dentro de la nueva columna creada

0

<ol>
    4. Se agrupa la tabla con base a la nueva columna <code>log_group</code> y determina: frecuencia absoluta, marca de clase, frecuencia relativa, frecuencia absoluta acumulada y frecuencia relativa acumulada:
</ol>

In [24]:
df0_gblg = df0[["LONGITUDE", "log_group"]].groupby("log_group").count()

df0_gblg.columns = ["lg_fi"]

df0_gblg["lg_class"] = round(df0[["LONGITUDE", "log_group"]].groupby("log_group").mean(),2)

log_total_count = df0_gblg["lg_fi"].sum()

df0_gblg["lg_hi"] = round(df0_gblg["lg_fi"]/log_total_count,4)

df0_gblg["lg_Fi"] = df0_gblg["lg_fi"].cumsum()

df0_gblg["lg_Hi"] = df0_gblg["lg_hi"].cumsum()

df0_gblg

,lg_fi,lg_class,lg_hi,lg_Fi,lg_Hi
log_group,,,,,
"(-164.0, -158.0)",15,-158.32,0.0003,15,0.0003
"(-158.0, -152.0)",102,-157.18,0.0021,117,0.0024
"(-152.0, -146.0)",43,-149.83,0.0009,160,0.0033
"(-140.0, -134.0)",6,-134.62,0.0001,166,0.0034
"(-134.0, -128.0)",2,-131.70,0.0000,168,0.0034
"(-128.0, -122.0)",2202,-122.48,0.0451,2370,0.0485
"(-122.0, -116.0)",5384,-118.85,0.1103,7754,0.1588
"(-116.0, -110.0)",1784,-112.57,0.0365,9538,0.1953
"(-110.0, -104.0)",1545,-105.80,0.0316,11083,0.2269


<p>Con esta tabla es posible decir que una gran parte de las instalaciones se encuentran ubicadas dentro del rango de longitud de <code>(-86.0, -80.0)</code>, es decir, 10.809 instalaciones están ubicadas en una longitud mayor o igual a -86.0 y menor que -80.0 según la columna <code>lg_fi</code>, y eso es equivalente a aproximadamente 22.14% del total de instalaciones de la compañía (columna <code>lg_hi</code>). También es posible concluir que, dada la columna <code>la_Fi</code>, 35.516 instalaciones se encuentran localizadas en una longitud mayor o igual a -164.0 y menor a -80.0, eso es equivalente al 72.73% de las instalaciones (columna <code>lg_Hi</code>).</p>

<p>Por lo que la tabla de frecuencias agrupadas también resulta útil para realizar diferentes observaciones con respecto a la distribución de los datos que se encuentran en la columna <code>LONGITUDE</code> y podemos ver el comportamiento de dicha variable y donde se centran dichos datos. Por ejemplo, si quisiéramos conocer qué porcentaje de instalaciones se encuentran en una longitud mayor o igual a -128.0 y menor que -68.0, entonces podríamos hacer lo siguiente:</p>

In [25]:
log_128_68 = (df0_gblg.loc[(-128.0, -122.0):(-74.0, -68.0),['lg_hi']].sum())*100 # Se secciona el índice con valores desde -128 hasta -68
                                                                                 # Y se cruzan con la columna 'lg_hi'
print("El porcentaje (%) de instalaciones cuya longitud está entre -128.0 y -68.0 es igual a: " + str(log_128_68))

El porcentaje (%) de instalaciones cuya longitud está entre -128.0 y -68.0 es igual a: lg_hi    99.63
dtype: float64


<p>Adicionalmente, si quisiéramos calcular el promedio de la variable <code>LONGITUDE</code> solamente es necesario multiplicar cada valor de la marca de clase (<code>lg_class</code>) con la frecuencia relativa (<code>lg_hi</code>) y sumar los productos:</p>

In [26]:
lg_mean = (df0_gblg["lg_class"]*df0_gblg["lg_hi"]).sum()
lg_mean

-91.121651

<p>Lo que quiere decir que las instalaciones tienden a estar localizadas en una longitud promedio aproximada de -91,12.</p>

## 3. Conclusión:

<p>Otro método para determinar las estadísticas más importantes es utilizando el método <code>.describe()</code> sobre los atributos <code>LATITUDE</code> y <code>LONGITUDE</code> así:</p>

<ol>
    1. Se aplica el método <code>.describe()</code> a la columna <code>LATITUDE</code>:
</ol>

In [27]:
df0["LATITUDE"].describe()

count    48821.000000
mean        37.206740
std          5.107545
min         19.445545
25%         33.715309
50%         38.406615
75%         40.900949
max         64.846461
Name: LATITUDE, dtype: float64

<p>Al aplicar el método anterior podemos determinar el número de valores (<code>count = 48821</code>), la media (<code>mean = 37,207</code>), la desviación estándar (<code>std = 5,108</code>), el valor mínimo (<code>min = 19,446</code>), el valor máximo (<code>max = 64,8464</code>) y otras medidas de locación (cuartiles). Y esto resulta muy útil debido a que nos brinda rápidamente información descriptiva clave sobre la variable, por ejemplo, la media calculada anteriormente (<code>la_mean</code>) es igual a 37,205 que es un valor muy aproximado a la media calculada a través del método '.describe()': 37,207.</p>

<p><b>Nota:</b> Cabe resaltar que el resultado obtenido con este método aplicado a una variable categórica como <code>STATE</code> es diferente. Bajo dichas condiciones, el método permite visualizar (aparte de <code>count</code>) cuántos valores únicos existen (<code>unique</code>), la moda (<code>top</code>), y la frecuencia de la moda (<code>freq</code>).</p>

<ol>
    2. Se aplica el método <code>.describe()</code> a la columna <code>LONGITUDE</code>:
</ol>

In [28]:
df0["LONGITUDE"].describe()

count    48821.000000
mean       -91.142779
std         16.193101
min       -159.502088
25%        -97.764952
50%        -86.174104
75%        -78.723784
max        -67.289981
Name: LONGITUDE, dtype: float64

<p>Lo mismo ocurre con la variable <code>LONGITUDE</code> al aplicar el método. La media calculada con el método de la tabla de frecuencia agrupada (-91.122) es supremamente similar a la obtenida a través de <code>.describe()</code> (-91.143), lo que le brinda confiabilidad y validez a este método de resumen de datos.</p>

<p>Finalmente, se guardan las tablas agrupadas con el fin de crear visualizaciones más adelante:</p>

In [29]:
PATH_1 = "DATASETS/DF_GB_LAT.csv"
PATH_2 = "DATASETS/DF_GB_LOG.csv"
df0_gbla.to_csv(PATH_1)
df0_gblg.to_csv(PATH_2)